In [205]:
import gym
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam 

In [191]:
env = gym.make("CartPole-v1")
env.reset()
n_actions = env.action_space.n
input_dim = env.observation_space.shape[0]

In [192]:
# nnet to approximate q 
model = Sequential()
# 5 = 4 states + 1 action
model.add(Dense(16, input_dim = 4 , activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(2, activation = 'linear'))
model.compile(optimizer=Adam(), loss = 'mse')

In [238]:
n_episodes = 12000
gamma = 0.95
epsilon = 0.1
minibatch_size = 32
r_sums = []
# replay memory holds s, a, r, s'
replay_memory = []
for n in range(n_episodes): 
    s = env.reset()
    done=False
    r_sum = 0
    while not done: 
        env.render()
        # 1. Feedforward pass for current state to get predicted q-values for all actions 
        qvals_s = model.predict(s.reshape(1,4))
        # 1a. choose action
        if np.random.random() < epsilon:  a = env.action_space.sample()
        else:                             a = np.argmax(qvals_s)
        sprime, r, done, info = env.step(a)
        # add to memory 
        if len(replay_memory) < 10000:
            replay_memory.append({"s":s,"a":a,"r":r,"sprime":sprime})
        else:
            replay_memory[n % len(replay_memory)] = {"s":s,"a":a,"r":r,"sprime":sprime}
        r_sum += r 
    # Experience Replay - update weights after every episode
    msize = min(minibatch_size,n)
    minibatch = np.random.choice(replay_memory, msize , replace=False)
    for i in range(msize): 
        tmp = minibatch[i]
        s,a,r,sprime = tmp['s'],tmp['a'],tmp['r'],tmp['sprime']
        # 2. Do a feedforward pass for the next state s’ and calculate 
        # maximum overall network outputs max a’ Q(s’, a’).
        qvals_sprime = model.predict(sprime.reshape(1,4))
        # 3. Set Q-value target for action to r + γmax a’ Q(s’, a’), 
        # For all other actions, set the Q-value target to the same 
        # as originally returned from step 1, making the error 0 for those outputs.
        target = r + gamma * np.max(qvals_sprime)
        target_f = model.predict(s.reshape((1,4)))
        target_f[0][a] = []
        for i in range(n_actions): 
            if i == a: target_arr.append(target)
            else:      target_arr.append(qvals_s[0][i])
        target_arr = np.array(target_arr)
        model.fit(s.reshape(1,4),target_arr.reshape(1,2), epochs=1, verbose=0)
    r_sums.append(r_sum)
    
    if n % 500 == 0 and n != 0 : 
        print(n) 
        print(np.mean(r_sums[n-100:n]))
        
        

500
9.78
1000
9.8
1500
10.0
2000
9.99
2500
9.77
3000
9.81
3500
9.92
4000
10.13
4500
9.74
5000
9.91
5500
9.87
6000
9.92
6500
9.88
7000
9.85
7500
10.04
8000
10.19
8500
9.7
9000
9.77
9500
9.81
10000
9.74
10500
9.83
11000
9.91
11500
9.82


In [ ]:
# Experience Replay - update weights after every episode
msize = min(minibatch_size,n)
minibatch = np.random.choice(replay_memory, msize , replace=False)
for i in range(msize): 
    tmp = minibatch[i]
    s,a,r,sprime = tmp['s'],tmp['a'],tmp['r'],tmp['sprime']
    # 2. Do a feedforward pass for the next state s’ and calculate 
    # maximum overall network outputs max a’ Q(s’, a’).
    qvals_sprime = model.predict(sprime.reshape(1,4))
    # 3. Set Q-value target for action to r + γmax a’ Q(s’, a’), 
    # For all other actions, set the Q-value target to the same 
    # as originally returned from step 1, making the error 0 for those outputs.
    target = r + gamma * np.max(qvals_sprime)
    target_arr = []
    for i in range(n_actions): 
        if i == a: target_arr.append(target)
        else:      target_arr.append(qvals_s[0][i])
    target_arr = np.array(target_arr)
    model.fit(s.reshape(1,4),target_arr.reshape(1,2), epochs=1, verbose=0)

In [239]:
minibatch = np.random.choice(replay_memory, msize , replace=False)


In [ ]:
targe

In [ ]:
minibatch = random.sample(self.memory, batch_size)
for s, a, r, sprime in minibatch:
   # target = r
   # if not done:
    target = r + gamma * np.amax(model.predict(sprime)[0])
    target_f = model.predict(s.reshape(1,4))
    target_f[0][a] = target
    model.fit(s, target_f, epochs=1, verbose=0)
if self.epsilon > self.epsilon_min:
    self.epsilon *= self.epsilon_decay

In [243]:
target_f = model.predict(s.reshape((1,4)))


In [244]:
target_f

array([[12929.587, 13232.123]], dtype=float32)

In [245]:
s

array([-0.04088862, -0.03568615,  0.0272934 , -0.02125022])